# Mundo 15 - Análise de sentimento de um texto com Python + CHATGPT

In [ ]:
from textblob import TextBlob
import requests
import json
from pprint import pprint
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from dotenv import load_dotenv
import os

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [ ]:
driver.get(f'https://ge.globo.com/futebol/times/flamengo/noticia/2023/04/20/jogadores-do-flamengo-se-empolgam-com-inicio-de-sampaoli-veja-depoimentos.ghtml')

In [ ]:
driver.get('https://ge.globo.com/futebol/times/flamengo/noticia/2023/04/06/roger-critica-vitor-pereira-acha-que-o-torcedor-do-flamengo-nao-entende-nada-e-cego-ou-e-bobo.ghtml')

In [ ]:
conteudo_ata = driver.find_element("xpath", '/html') 

html_ata = conteudo_ata.get_attribute('outerHTML')

#driver.quit()

soup = BeautifulSoup(html_ata, 'html.parser')

paragrafos = soup.find_all("p", class_ = 'content-text__container') 

paragrafos

In [ ]:
paragrafos[1].text

In [ ]:
load_dotenv() 

In [ ]:
token = os.environ.get("openai_token")

In [ ]:
headers = {'Authorization': token,
          'Content-Type': 'application/json'}

In [ ]:
response = requests.get('https://api.openai.com/v1/models', headers = headers)

pprint(response.json())

In [ ]:
id_modelo = 'gpt-3.5-turbo'

texto_traduzido = []

In [ ]:
for texto in paragrafos:
    
    mensagem_gpt = {'model': id_modelo,
                   'messages': [{'role': 'user',
                                'content': "Traduza o seguinte texto para o inglês, me retornando APENAS a tradução:" + texto.text}]}


    json_mensagem = json.dumps(mensagem_gpt)
    
    
    response = requests.post('https://api.openai.com/v1/chat/completions', 
                                    headers = headers, data = json_mensagem)
    
    texto_traduzido.append(response)
    
    time.sleep(15)

In [ ]:
texto_final = ''

for texto in texto_traduzido:
    
    if texto.status_code == 200:
        
        parte_traduzida = texto.json()['choices'][0]['message']['content']

        texto_final += parte_traduzida


texto_final

In [ ]:
blob = TextBlob(texto_final)

In [ ]:
sentimento = blob.sentiment.polarity #espectro de -1 a 1

In [ ]:
sentimento

In [ ]:
0.37185425685425694